# Analyse de l'Évolution de la Température Moyenne Cumulative par Année


L'analyse présentée dans ce notebook s'intéresse à l'évolution de la température moyenne cumulative par année, en se basant sur des données météorologiques quotidiennes enregistrées par divers postes météorologiques en France. Les données utilisées proviennent du fichier `Q_34_previous-1950-2023_RR-T-Vent.csv.gz`, disponible à l'adresse https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_34_previous-1950-2023_RR-T-Vent.csv.gz. Ce fichier contient des informations sur les conditions météorologiques telles que les précipitations, les températures minimales et maximales, la vitesse du vent, etc.


L'objectif de cette analyse est de cumuler la température moyenne journalière par une somme cumulative depuis le début de chaque année pour toutes les stations, tout en ajoutant une colonne représentant l'opacité qui augmente avec les années. Cela nous permettra de visualiser l'évolution de la température moyenne cumulative au fil des ans et de mettre en évidence les tendances éventuelles.


## Méthodologie


La méthodologie utilisée dans cette analyse consiste à appliquer une requête SQL sur les données chargées pour calculer les températures moyennes quotidiennes, puis à cumuler ces températures par année et jour de l'année. Les étapes clés de cette méthodologie incluent le filtrage des données pour ne conserver que les enregistrements avec des températures moyennes valides, le calcul de la température moyenne quotidienne, et enfin, le cumul de ces températures par année. La requête SQL générée permet de réaliser ces étapes de manière efficace.


Les résultats de cette analyse sont ensuite visualisés à l'aide d'une bibliothèque de visualisation de données, permettant de représenter graphiquement l'évolution de la température moyenne cumulative par année. La visualisation obtenue permet d'apprécier les variations de température d'une année à l'autre et de détecter d'éventuelles tendances à long terme.

## 🔧 Configuration

In [1]:
# Installation et imports
import duckdb as ddb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## 🦆 Chargement du dataset avec Duckdb

In [2]:
# Fonction de chargement complète (basée sur load_file_from_url_lite)
def load_file_from_url_lite(url_dataset="", loader="read_csv_auto", options="", nom_table="loaded_dataset", safe_mode=False):
    ddb.execute("install spatial")
    ddb.execute("load spatial")
    ddb.execute("INSTALL h3 FROM community")
    ddb.execute("LOAD h3")
    ddb.execute("install webbed from community;")
    ddb.execute("load webbed")
    ddb.execute("set force_download=True")
    ddb.execute(f"drop table if exists {nom_table}")   
    
    # Détection automatique du type de fichier
    if 'csv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'tsv' in url_dataset: 
        loader = "read_csv_auto"
    elif 'txt' in url_dataset: 
        loader = "read_csv_auto"
    elif 'parquet' in url_dataset: 
        loader = "read_parquet"
    elif 'json' in url_dataset: 
        loader = "read_json_auto"
    elif 'xls' in url_dataset or 'xlsx' in url_dataset: 
        loader = "st_read"
    elif 'shp' in url_dataset: 
        loader = "st_read"
    elif 'geojson' in url_dataset: 
        loader = "st_read"
    elif 'xml' in url_dataset: 
        loader = "read_xml"
    elif 'html' in url_dataset: 
        loader = "read_html"
    else: 
        raise ValueError(f"Type de fichier non supporté pour {url_dataset}")
    
    if options=="": 
        options = "" 
    if 'csv' in url_dataset and safe_mode==True: 
        options = ", all_varchar=1" 
    if nom_table=="": 
        nom_table = "loaded_dataset"
    
    try:
        status = ddb.sql(f"""
            create or replace table {nom_table} as select *
            from
            {loader}("{url_dataset}" {options})
        """)
        return status
    except Exception as e:
        return f"Erreur au chargement du fichier : {str(e)}"

def run_query(sql):
    return ddb.sql(sql.replace("`"," ")).to_df()

# Chargement des données
load_file_from_url_lite("https://object.files.data.gouv.fr/meteofrance/data/synchro_ftp/BASE/QUOT/Q_34_previous-1950-2023_RR-T-Vent.csv.gz", safe_mode=True)
print("✅ Données chargées avec succès")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Données chargées avec succès


## 🔍 Analyse SQL

Cette requête utilise des techniques SQL pour extraire et transformer les données de manière efficace.

In [3]:
# Exécution de la requête
df = run_query(""" WITH daily_avg AS (
  SELECT 
    CAST(SUBSTRING("AAAAMMJJ", 1, 4) AS INTEGER) AS annee,
    CAST(SUBSTRING("AAAAMMJJ", 5, 2) AS INTEGER) AS mois,
    CAST(SUBSTRING("AAAAMMJJ", 7, 2) AS INTEGER) AS jour,
    CAST("TM" AS DOUBLE) AS tm
  FROM loaded_dataset
  WHERE "TM" IS NOT NULL AND "QTM" = '1'
),
annual_cumulative AS (
  SELECT 
    annee,
    jour_de_lannee,
    AVG(tm) AS avg_tm,
    SUM(AVG(tm)) OVER (PARTITION BY annee ORDER BY jour_de_lannee) AS cumulative_tm
  FROM (
    SELECT 
      annee,
      (mois - 1) * 30 + jour AS jour_de_lannee,
      tm
    FROM daily_avg
  ) sub
  GROUP BY annee, jour_de_lannee
)
SELECT 
  annee,
  jour_de_lannee,
  cumulative_tm,
  (annee - MIN(annee) OVER ()) / (MAX(annee) OVER () - MIN(annee) OVER ()) AS opacite
FROM annual_cumulative
ORDER BY annee, jour_de_lannee """)
print(f"Résultats : {len(df)} lignes")
df.head()

Résultats : 26584 lignes


,annee,jour_de_lannee,cumulative_tm,opacite
0,1950,1,6.70,0.0
1,1950,2,11.25,0.0
2,1950,3,19.55,0.0
3,1950,4,30.65,0.0
4,1950,5,40.50,0.0


## 📈 Visualisation

La bibliothèque principale utilisée est Plotly, qui permet de créer des visualisations interactives et dynamiques. Cette technologie est adaptée pour représenter l'évolution de la température moyenne cumulative par année de manière claire et détaillée, avec des fonctionnalités de survol pour afficher les informations précises. Cela facilite la comparaison entre les différentes années.

In [4]:
import pandas as pd
import duckdb as ddb
import plotly.graph_objects as go
import pandas as pd

# Calcul des moyennes et cumul
df_moyen = df.groupby(['annee','jour_de_lannee']).cumulative_tm.mean().reset_index()
df_moyen['cumulative_sum'] = df_moyen.groupby('annee')['cumulative_tm'].cumsum()
min_y, max_y = df_moyen.annee.min(), df_moyen.annee.max()
df_moyen['opacite'] = (df_moyen.annee - min_y)/(max_y - min_y)

dataviz = go.Figure()
for annee in sorted(df_moyen.annee.unique()):
    tmp = df_moyen[df_moyen.annee==annee]
    color = 'red' if annee==2022 else 'rgba(128,128,128,'+str(tmp.opacite.iloc[0])+')'
    dataviz.add_trace(go.Scatter(x=tmp.jour_de_lannee, y=tmp.cumulative_sum,
                                mode='lines', line=dict(color=color),
                                hovertemplate=f'<b>{annee}</b><br>Jour:%{{x}}<br>Cumul:%{{y:.1f}}<extra></extra>'))
dataviz.update_layout(title="Évolution de la Température Moyenne Cumulative par Année",
                      xaxis_title='Jour de l\'année', yaxis_title='Température cumulative',
                      hovermode='closest', 
                      margin=dict(l=20, r=20, t=40, b=20))
dataviz

---
*Made with ❤️ and with [duckit.fr](https://duckit.fr) - [Ali Hmaou](https://www.linkedin.com/in/ali-hmaou-6b7b73146/)*

In [5]:

# --- Variables injectées par le script ---
FINAL_OBJECT_VARIABLE_NAME = 'dataviz'
OUTPUT_IMAGE_NAME = 'published\\notebooks\\duckit_analysis_20250820_212152.png'
OUTPUT_HTML_NAME = 'published\\notebooks\\duckit_analysis_20250820_212152.html'

# ===================================================================
# CELLULE INJECTÉE AUTOMATIQUEMENT (VERSION ROBUSTE)
# ===================================================================
import sys
import os
# On importe les modules nécessaires pour l'export au cas où
try:
    from bokeh.io import save as bokeh_save
except ImportError:
    bokeh_save = None

try:
    # On s'assure que le dossier de sortie existe
    output_dir = os.path.dirname(OUTPUT_IMAGE_NAME)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    # On utilise globals().get() pour une récupération plus sûre
    final_object = globals().get(FINAL_OBJECT_VARIABLE_NAME)

    if final_object is None:
        # On lève une NameError pour être cohérent avec le code original
        raise NameError(f"name '{FINAL_OBJECT_VARIABLE_NAME}' is not defined")

    print(f"INFO: Variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée. Tentative d'exportation...")

    object_type = str(type(final_object))

    if 'plotly.graph_objs._figure.Figure' in object_type:
        print(f"--> Détecté : Plotly. Sauvegarde HTML et PNG.")
        # 1. Sauvegarde HTML pour l'interactivité
        print(f"--> Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.write_html(OUTPUT_HTML_NAME, include_plotlyjs='cdn')
        # 2. Sauvegarde PNG pour l'aperçu statique
        try:
            print(f"--> Tentative de sauvegarde PNG directe dans : {OUTPUT_IMAGE_NAME}")
            final_object.write_image(OUTPUT_IMAGE_NAME, scale=3, width=1200, height=800)
            print(f"--> Image Plotly sauvegardée avec succès.")
        except Exception as e:
            print(f"AVERTISSEMENT: La sauvegarde directe en PNG a échoué (kaleido est-il installé?).", file=sys.stderr)
            print(f"   Erreur: {e}", file=sys.stderr)
            print(f"--> PLAN B: On va utiliser la capture d'écran du HTML à la place.")
            # On crée un fichier marqueur pour que le script de post-traitement prenne le relais
            with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
                f.write("plotly")
    elif 'folium.folium.Map' in object_type:
        print(f"--> Détecté : Folium. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("folium")
    elif 'altair.vegalite' in object_type and hasattr(final_object, 'save'):
        print(f"--> Détecté : Altair. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        final_object.save(OUTPUT_HTML_NAME)
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("altair")
    elif 'bokeh.plotting' in object_type and bokeh_save is not None:
        print(f"--> Détecté : Bokeh. Sauvegarde HTML dans : {OUTPUT_HTML_NAME}")
        bokeh_save(final_object, filename=OUTPUT_HTML_NAME, title="")
        # On crée un fichier marqueur générique pour la capture d'écran
        print(f"--> Création du marqueur de capture d'écran.")
        with open(f"{OUTPUT_HTML_NAME}.needs_screenshot", "w") as f:
            f.write("bokeh")
    elif 'matplotlib.figure.Figure' in object_type:
        print(f"--> Détecté : Matplotlib. Sauvegarde dans : {OUTPUT_IMAGE_NAME}")
        final_object.savefig(OUTPUT_IMAGE_NAME, dpi=300, bbox_inches='tight')
    else:
        print(f"AVERTISSEMENT: Type non supporté : {object_type}", file=sys.stderr)
except NameError:
    print(f"AVERTISSEMENT: Aucune variable '{FINAL_OBJECT_VARIABLE_NAME}' trouvée.", file=sys.stderr)
except Exception as e:
    print(f"ERREUR lors de l'exportation : {e}", file=sys.stderr)


INFO: Variable 'dataviz' trouvée. Tentative d'exportation...
--> Détecté : Plotly. Sauvegarde HTML et PNG.
--> Sauvegarde HTML dans : published\notebooks\duckit_analysis_20250820_212152.html
--> Tentative de sauvegarde PNG directe dans : published\notebooks\duckit_analysis_20250820_212152.png


--> Image Plotly sauvegardée avec succès.
